# Boltzmann portfolios

### Part 2: Multi-period environment

We develop an alternative to the traditional mean-variance
framework ("*Markowitz*" portfolios) called
***Boltzmann*** portfolios which addresses uncertainty
from the standpoint of entropy and optimal sequential decisions.
The result is a faster online algorithm which is more robust
and has no dependencies on offline convex optimization packages.

Markowitz portfolios are optimal in the arithmetic mean-variance
framework for a *single-period*. They are fragile to changing
market conditions, much like elegant battle strategies which
crumble under harsh war conditions.
In contrast, Boltzmann portfolios are designed to be
adaptive over multiple periods to maximize final wealth.
Techniques have been borrowed from Bayesian and
reinforcement learning.

[Sequential decisions to be covered in Part ?.]

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235
     
*CHANGE LOG*

    2017-06-28  Functions moved to ys_prtf_boltzmann module.
    2017-06-27  First version, functions numerically tested.

In [2]:
from fecon235.fecon235 import *

In [3]:
#  PREAMBLE-p6.15.1223d :: Settings and system details
from __future__ import absolute_import, print_function, division
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

 ::  Python 2.7.13
 ::  IPython 5.1.0
 ::  jupyter_core 4.2.1
 ::  notebook 4.1.0
 ::  matplotlib 1.5.1
 ::  numpy 1.11.0
 ::  scipy 0.17.0
 ::  sympy 1.0
 ::  pandas 0.19.2
 ::  pandas_datareader 0.2.1
 ::  Repository: fecon235 v5.17.0603 devPrtf
 ::  Timestamp: 2017-06-28T23:40:26Z
 ::  $pwd: /media/yaya/virt15h/virt/dbx/Dropbox/ipy/fecon235/nb


## Introduction

***It is important to understand Part 1 of this series.
We begin by condensing its content to code,
after constructing a dataframe.***

## Download data and construct a dataframe

We retrieve the following data of daily frequency
representing equities worldwide and gold by five ETF securities: 

In [4]:
#  Convenient dictionary set in fecon235.py,
#  where key is world region, and value is its fecon235 data code:
world4d

{'America': 's4spy',
 'Emerging': 's4eem',
 'Europe': 's4ezu',
 'Gold': 's4gld',
 'Japan': 's4ewj'}

In [5]:
#  Or manually specify your own dictionary here:
prices_dic = world4d

In [6]:
#  Download data into a dataframe, alphabetically by key:
prices = groupget( prices_dic, maxi=3650 )
#  ... about ten years worth.

 ::  Retrieved from Google Finance: SPY
 ::  Retrieved from Google Finance: EEM
 ::  Retrieved from Google Finance: EZU
 ::  Retrieved from Google Finance: GLD
 ::  Retrieved from Google Finance: EWJ


## Extract the geometric mean rates

A Boltzmann portfolio seeks the best, not necessarily maximal,
weighted geometric mean rate of its underlying assets.

In [7]:
rates = gemratarr( prices )
rates

array([[  2.09],
       [-11.07],
       [-10.4 ],
       [  4.05],
       [ -5.01]])

## Global weights from covariance matrix

We now turn our attention to the weights associated with the **Global
Minimum Variance Portfolio**. Its derivation is found in Cochrane
(2005), chp. 5, p.83:

$$ \mathbf{w} = \frac{V^{-1}\mathbf{1}} { \mathbf{1}^\top V^{-1} \mathbf{1} } $$

Note that the weights are solely dependent on the covariance matrix $V$.
There are no constraints involved.

In [9]:
weighcovdata?

Signature: weighcovdata(dataframe)
Docstring: WEIGHT array (N,1) for Global Min Var Portfolio, given data.
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [10]:
globalw = weighcovdata( prices )
globalw

array([[ 0.9033044 ],
       [-0.34947725],
       [-0.19632884],
       [ 0.4654879 ],
       [ 0.17701379]])

## Trim global weights

Negative weights imply the underlying assets should be shorted.
A Boltzmann portfolio only considers the weights as ***advisory***.

We may want to limit short sales at -0.3 weight, or perhaps ignore tiny
positions for rebalancing purposes in a multiple-period setting.

So we specify a threshold weight, and renormalize.

In [11]:
MIN_weight = 0.05

In [12]:
weights = rentrim( globalw, MIN_weight, 0 )
weights

array([[ 0.58435816],
       [ 0.        ],
       [ 0.        ],
       [ 0.30112956],
       [ 0.11451229]])

In [13]:
scores = weights * rates
scores

array([[ 1.22130855],
       [-0.        ],
       [-0.        ],
       [ 1.2195747 ],
       [-0.57370656]])

In [14]:
#  Portfolio return with trimmed weights:
np.sum(scores)

1.8671766914559962

## Introducing Boltzmann and softmax

In the field of reinforcement learning, the softmax function is used to
convert values into action probabilities.
A positive parameter T called the temperature is introduced
to divide through each value. It is a scaling operation such that
high temperatures cause corresponding actions to be equi-probable.
Low temperatures cause a greater difference in selection probability
for actions that differ in their value estimates.
At low temperatures, the probability of the action with the
highest expected reward tends to 1.

In [16]:
weighsoft??

Signature: weighsoft(weights, rates, temp, floor, level)
Source:   
def weighsoft( weights, rates, temp, floor, level ):
    '''Compute new weights transformed by softmax function.'''
    scores = weights * rates
    problist = mlearn.softmax( scores, temp )[-1]
    probs = np.array( problist ).reshape(len(problist), 1)
    #  Revise weights based on softmax probabilities:
    pweights = probs * weights
    #  Then appropriately adjust:
    return rentrim(renormalize(pweights), floor, level)
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [17]:
#  The temperature is a hyperparameter which should be varied:
TEMPERATURE = 55

In [18]:
pweights = weighsoft(weights, rates, TEMPERATURE, MIN_weight, 0)
pweights

array([[ 0.6607545],
       [ 0.       ],
       [ 0.       ],
       [ 0.3392455],
       [ 0.       ]])

In [19]:
scores_soft = pweights * rates
scores_soft

array([[ 1.38097691],
       [-0.        ],
       [-0.        ],
       [ 1.37394427],
       [-0.        ]])

In [20]:
#  Portfolio return for Boltzmann portfolio:
np.sum(scores_soft)

2.7549211782620033

## Unify functions


```
    prices ---> cov ---> globalw
      |                    |
      |                  trimit  <-- floor
      |                  renormalize
      |                    |
      v                    v
      |                    |
    gemrat              weights
      |                    |
      |________score_______|
                 |
                 |                   Boltzmann
      temp --> softmax --> probs --> pweights

             
```

In [21]:
boltzweigh??

Signature: boltzweigh(dataframe, yearly=256, temp=55, floor=0.01, level=0)
Source:   
def boltzweigh(dataframe, yearly=256, temp=55, floor=0.01, level=0):
    '''MAIN: Compute softmax weights of a Boltzmann portfolio.'''
    rates = gemratarr(dataframe, yearly)
    globalw = weighcovdata(dataframe)
    weights = rentrim(globalw, floor, level)
    return weighsoft(weights, rates, temp, floor, level)
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [22]:
boltzweigh( prices, temp=TEMPERATURE, floor=MIN_weight )
#  should equal pweights above.

array([[ 0.6607545],
       [ 0.       ],
       [ 0.       ],
       [ 0.3392455],
       [ 0.       ]])

In [30]:
boltzportfolio( prices, temp=TEMPERATURE, floor=MIN_weight )

America 0.6608 est. 2.09 rate
Emerging 0.0 est. -11.07 rate
Europe 0.0 est. -10.4 rate
Gold 0.3392 est. 4.05 rate
Japan 0.0 est. -5.01 rate
_________
PORTFOLIO geometric mean return: 2.75 %


---

## References

- John H. Cochrane, 2005 revised ed., *Asset Pricing*, Princeton U. Press.

- On softmax:
    - https://en.wikipedia.org/wiki/Softmax_function 
    - http://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative
    - http://cs231n.github.io/linear-classify/#softmax 
    - https://en.wikipedia.org/wiki/Reinforcement_learning 

# TESTS

In [31]:
for T in range(1, 421, 25):
    bw = boltzweigh( prices, temp=T, floor=-10, level=-10 )
    bwr = roundit( list(bw.flat), 2, echo=False )
    print( T, "\t", bwr )

1 	 [0.0, -1.0, -0.0, 0.0, 0.0]
26 	 [0.08, -1.09, -0.02, 0.04, 0.0]
51 	 [0.86, -2.1, -0.22, 0.44, 0.01]
76 	 [0.89, -0.19, -0.19, 0.46, 0.03]
101 	 [2.82, -2.77, -0.66, 1.45, 0.15]
126 	 [1.86, -1.51, -0.43, 0.96, 0.13]
151 	 [1.54, -1.11, -0.35, 0.79, 0.13]
176 	 [1.39, -0.91, -0.31, 0.71, 0.13]
201 	 [1.29, -0.8, -0.29, 0.67, 0.13]
226 	 [1.23, -0.72, -0.28, 0.63, 0.13]
251 	 [1.18, -0.67, -0.26, 0.61, 0.14]
276 	 [1.15, -0.62, -0.26, 0.59, 0.14]
301 	 [1.12, -0.59, -0.25, 0.58, 0.14]
326 	 [1.1, -0.57, -0.24, 0.57, 0.14]
351 	 [1.08, -0.55, -0.24, 0.56, 0.15]
376 	 [1.07, -0.53, -0.24, 0.55, 0.15]
401 	 [1.06, -0.52, -0.23, 0.54, 0.15]
